In [ ]:
import random

In [ ]:
class Individual:
    def __init__(self, data, capacity, ...):
        self.solution = self.generate_solution()
        self. fitness = self.calc_fitness()

    def generate_solution(self):
        #kako kodirati hromozom? - moze npr u jednom nizu gde ide prvo redni broj vozila na kojise doda broj customera pa onda redni brojevi customera koje obilazi na svojoj ruti pa onda naredno vozilo itd
        #moraju da se provere svi uslovi, u zavisnosti da li radimo klasican vrp ili sa time windowom 
        code = []
        while(not self.is_feasible(code)):
            pass

        return code

    def calc_fitness():
        #da li ce fitness biti predjeni put ili vreme, manje vise je sve jedno 
        #ali ako bismo koristili neke vremenske varijable, npr da uracunamo u cost i vreme istovara robe ili tako nesto, ili ako gledamo time windowe pa moramo racunati vreme
        #a pritom imamo samo distance izmedju cvorova, to bi onda morali da prevedemo na vreme puta, npr predjeni put podelimo sa nekom prosecnom brzinom za koju se dogovorimo osim ako ne nadjemo neki dobar dataset 
        fitness = 0``
        return fitness



    def is_feasible(self, code):
        # provera uslova ovo od ranije 




In [ ]:
def selection(population, tournament_size):
    tournament = random.sample(population, tournament_size)
    return max(tournament, key = lambda x: x.fitness)

In [ ]:
def crossover(parent1, parent2, child1, child2):
    #ovde mozemo da probamo sa vise crossovera, videla sam neki kao order crossover koji mi deluje bas ok ali ima ih jos ali ne znam kolko bi to uticalo na poboljsanje resenja 
    

In [ ]:
def mutation(individual, mutation_prob):
    # mutacija moze da bude jednostavna, samo se izberu dve rute i swapuje se po jedan customer iz svake 

In [ ]:
def genetic_algorithm(data, population_size, elitism_size, mutation_prob, tournament_size, num_generations, capacity, ...):
    population = [Individual(data, capacity, ...) for _ in range(population_size)]
    new_population = population.copy()

    for i in range(num_generations):
        population.sort(key = lambda x: x.fitness, reverse = True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            while(parent1 == parent2):
                parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1= new_population[j], child2 = new_population[j+1])
            
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)

            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()

        population = new_population.copy()
    
    return max(population, key = lambda x: x.fitness)